<a href="https://colab.research.google.com/github/mwojcik01/CNN/blob/main/!praca_!Gotowa_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#kopiowanie
import os
import numpy as np
import shutil

#przygotowanie modelu
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras import layers
from keras import optimizers
#from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

#wykres
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!unzip -uq "/content/drive/My Drive/base_ddsm_cbm_2.zip" -d "./" # wypakowanie plików z Drive

In [ ]:
base_dir = './base_ddsm_cbm_2' # obliczanie ilości plików w każdym folderze
raw_no_of_files = {}
classes = ['ben_mal', 'nor']
for dir in classes:
    raw_no_of_files[dir] = len(os.listdir(os.path.join(base_dir, dir)))

raw_no_of_files.items()

dict_items([('ben_mal', 2003), ('nor', 9215)])

In [ ]:
#kasowanie plików
%%bash
pwd
rm -R images
ls

/content
base_ddsm_cbm_2
drive
sample_data


rm: cannot remove 'images': No such file or directory


In [ ]:
data_dir = './images' # tworzenie katalogów: treningowego, walidacyjnego oraz testowego. 
                      #w każdym znajdują się 2 foldery: nor, ben_mal

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

train_nor_dir = os.path.join(train_dir, 'nor')
train_ben_mal_dir = os.path.join(train_dir, 'ben_mal')

valid_nor_dir = os.path.join(valid_dir, 'nor')
valid_ben_mal_dir = os.path.join(valid_dir, 'ben_mal')

test_nor_dir = os.path.join(test_dir, 'nor')
test_ben_mal_dir = os.path.join(test_dir, 'ben_mal')

for directory in (train_dir, valid_dir, test_dir):
    if not os.path.exists(directory):
        os.mkdir(directory)

dirs = [train_nor_dir, train_ben_mal_dir, valid_nor_dir, valid_ben_mal_dir, test_nor_dir, test_ben_mal_dir]

for dir in dirs:
    if not os.path.exists(dir):
        os.mkdir(dir)

In [ ]:
#kopiowanie
nor_fnames = os.listdir(os.path.join(base_dir, 'nor'))
ben_mal_fnames = os.listdir(os.path.join(base_dir, 'ben_mal'))


nor_fnames = [fname for fname in nor_fnames if fname.split('.')[1].lower() in ['png']]
ben_mal_fnames = [fname for fname in ben_mal_fnames if fname.split('.')[1].lower() in ['png']]

In [ ]:
size = min(len(nor_fnames), len(ben_mal_fnames)) # ustalanie ilości plików w folderze 

train_size = int(np.floor(0.7 * size))
valid_size = int(np.floor(0.2 * size))
test_size = size - train_size - valid_size

train_idx = train_size
valid_idx = train_size + valid_size
test_idx = train_size + valid_size + test_size

In [ ]:
for i, fname in enumerate(nor_fnames): # proces kopiowania plików
    if i <= train_idx:
        src = os.path.join(base_dir, 'nor', fname)
        dst = os.path.join(train_nor_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'nor', fname)
        dst = os.path.join(valid_nor_dir, fname)
        shutil.copyfile(src, dst) 
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'nor', fname)
        dst = os.path.join(test_nor_dir, fname)
        shutil.copyfile(src, dst) 

for i, fname in enumerate(ben_mal_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'ben_mal', fname)
        dst = os.path.join(train_ben_mal_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'ben_mal', fname)
        dst = os.path.join(valid_ben_mal_dir, fname)
        shutil.copyfile(src, dst) 
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'ben_mal', fname)
        dst = os.path.join(test_ben_mal_dir, fname)
        shutil.copyfile(src, dst)

In [ ]:
# sprawdzanie ilości plików w folderach
print('nor - trening', len(os.listdir(train_nor_dir))) 
print('nor - walidacja', len(os.listdir(valid_nor_dir)))
print('nor - test', len(os.listdir(test_nor_dir)))

print('ben_mal - trening', len(os.listdir(train_ben_mal_dir)))
print('ben_mal - walidacja', len(os.listdir(valid_ben_mal_dir)))
print('ben_mal - test', len(os.listdir(test_ben_mal_dir)))

nor - trening 1403
nor - walidacja 400
nor - test 200
ben_mal - trening 1403
ben_mal - walidacja 400
ben_mal - test 200


In [ ]:
# przeskalowanie obrazów o współczynnik 1/255
train_datagen = ImageDataGenerator(rescale=1./255.)
valid_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                   target_size=(128, 128),
                                                   batch_size=20,
                                                   class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(directory=valid_dir,
                                                   target_size=(128, 128),
                                                   batch_size=20,
                                                   class_mode='binary')

Found 2806 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [ ]:
for data_batch, labels_batch in train_generator:
    print('dane:', data_batch.shape)
    break

dane: (20, 128, 128, 3)


In [ ]:
model = Sequential(
    [
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(128, 128, 3)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(units=256, activation='relu'),
        layers.Dense(units=1, activation='sigmoid') 
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        3

In [ ]:
batch_size = 16
steps_per_epoch = train_size // batch_size
validation_steps = valid_size // batch_size

In [ ]:
model.compile(optimizer='Adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

history = model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=30,
                    validation_data=valid_generator,
                    validation_steps=validation_steps)

Epoch 1/30
87/87 [==============================] - 34s 35ms/step - loss: 0.6963 - accuracy: 0.5087 - val_loss: 0.6921 - val_accuracy: 0.5400
Epoch 2/30
87/87 [==============================] - 3s 30ms/step - loss: 0.6938 - accuracy: 0.5098 - val_loss: 0.6907 - val_accuracy: 0.4820
Epoch 3/30
87/87 [==============================] - 3s 30ms/step - loss: 0.6706 - accuracy: 0.5684 - val_loss: 0.6436 - val_accuracy: 0.6380
Epoch 4/30
87/87 [==============================] - 3s 31ms/step - loss: 0.5660 - accuracy: 0.7155 - val_loss: 0.5666 - val_accuracy: 0.7000
Epoch 5/30
87/87 [==============================] - 3s 30ms/step - loss: 0.5380 - accuracy: 0.7341 - val_loss: 0.5213 - val_accuracy: 0.7380
Epoch 6/30
87/87 [==============================] - 3s 30ms/step - loss: 0.5339 - accuracy: 0.7399 - val_loss: 0.5242 - val_accuracy: 0.7480
Epoch 7/30
87/87 [==============================] - 3s 30ms/step - loss: 0.5240 - accuracy: 0.7520 - val_loss: 0.5339 - val_accuracy: 0.7460
Epoch 8/30
8

In [ ]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Accuracy vs. Val Accuracy', xaxis_title='Epoki', yaxis_title='Accuracy', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Loss vs. Val Loss', xaxis_title='Epoki', yaxis_title='Loss', yaxis_type='log')
    fig.show()

plot_hist(history)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                 target_size=(128, 128),
                                                 batch_size=8,
                                                 class_mode='binary')

test_loss, test_acc = model.evaluate(test_generator, steps=50)
print('Dokładność testowania:', test_acc)

Found 400 images belonging to 2 classes.
50/50 [==============================] - 1s 9ms/step - loss: 0.6847 - accuracy: 0.7950
Dokładność testowania: 0.7950000166893005


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=1,
    class_mode='binary',
    shuffle=False
)

y_prob = model.predict(test_generator, test_generator.samples)
y_prob = y_prob.ravel()
y_prob

Found 400 images belonging to 2 classes.


array([9.76387858e-01, 9.92750004e-03, 1.36457346e-02, 1.96382659e-03,
       5.34074724e-01, 9.92725404e-08, 2.04148367e-02, 2.07998864e-02,
       4.42126766e-03, 2.36216920e-06, 2.02148817e-02, 9.49158430e-01,
       2.13702442e-08, 2.52203830e-03, 5.90719618e-02, 6.00390085e-05,
       2.84630200e-03, 3.22169578e-03, 4.87764794e-07, 6.01206557e-06,
       3.38540273e-03, 3.65002692e-04, 1.02375997e-02, 3.22639823e-01,
       5.25367128e-09, 3.90974674e-06, 6.11770565e-06, 3.15049765e-05,
       1.38249070e-05, 3.90285417e-03, 2.28309782e-05, 1.36334511e-05,
       2.93782045e-06, 3.83224119e-09, 2.52931747e-12, 2.25160271e-02,
       9.98983562e-01, 4.14921492e-02, 1.51616605e-05, 1.93270098e-04,
       3.66749167e-01, 4.32860315e-01, 5.05907228e-04, 2.10063890e-01,
       8.84100676e-01, 5.21308064e-01, 1.18822902e-01, 1.69380040e-13,
       5.36508441e-01, 1.46836073e-05, 1.02282909e-03, 3.12850042e-03,
       1.01714089e-04, 3.10545642e-04, 6.10568514e-03, 1.44005810e-06,
      

In [ ]:
predictions  = pd.DataFrame({'y_prob': y_prob})
predictions['class'] = predictions['y_prob'].apply(lambda x: 1 if x > 0.5 else 0)
predictions

,y_prob,class
0,0.976388,1
1,0.009928,0
2,0.013646,0
3,0.001964,0
4,0.534075,1
...,...,...
395,1.000000,1
396,1.000000,1
397,0.994960,1
398,0.877201,1


In [ ]:
y_true = test_generator.classes
y_true

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
y_pred = predictions['class'].values
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,

In [ ]:
test_generator.class_indices

{'ben_mal': 0, 'nor': 1}

In [ ]:
cm = confusion_matrix(y_true, y_pred)
cm

array([[156,  44],
       [ 38, 162]])

In [ ]:
def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=classes, index=classes[::-1])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=500, height=500, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)